## Bon alors c'est quoi c't histoire de package

*ctrl_nmod* Ouais le nom est pourri j'en trouverai un mieux plus tard

C'est basé sur un peu la même architecture que PyTorch parce que c'est les mêmes besoins.

Ce qui vous intéresse se trouve dans deux endroits : 
* ctrl_nmod/models/ssmodels/grnssm.py presque le modèle d'état le plus général qu'on peut concevoir 
* ctrl_nmod/utils/data.py et les classes **Experiments** et **ExperimentsDataset**


**Installation**

Normalement si j'ai pas tout cassé

``` pip install -e . ```

devrait faire le job

***Experiments***

L'idée étant d'avoir une structure de données compatible avec un jeu de données composés de plusieurs trajectoires différentes (pas nécessairement de même longueur)

Une **Experiment** est composé de (u,y) et éventuellement x si on a accès à l'état. 
Si on ne mesure pas l'état alors on crée un vecteur x du même longueur que u ou y.

Dans ce cas on doit fournir la taille **nx** du vecteur d'état.

On a ensuite le choix de rendre ce nouveau vecteur **trainable** ou pas c'est à dire de l'inclure dans le problème d'optimisation.

On piochera dans cette expérience une séquence de **seq_len** de quadruplets (u, y, x, x0).

La structure se débrouille pour ne pas prendre les **seq_len** derniers éléments de l'expérience.

<img src="experiments.png" alt="Exemple experiments" style="width:50%; height:auto;">

**ExperimentsDataset**

C'est une classe qui implémente une liste d'*Experiments* et qui gère les indices avec la mise en batch.

On peut donc ajouter une expérience au dataset avec la méthode append()

In [1]:
from ctrl_nmod.utils.data import ExperimentsDataset, Experiment
from data.pendulum.load_pendulum import load_pendulum


u_train, y_train, u_test, y_test, ts = load_pendulum(['data_train_francois.mat', 'data_test_francois.mat'])
u_train_yuqi, y_train_yuqi, u_test_yuqi, y_test_yuqi, ts = load_pendulum(['data_train_yuqi.mat', 'data_test_yuqi.mat'])

# Length of sequences to consider
seq_len = 20
nx = 2

train_set = ExperimentsDataset([Experiment(u_train, y_train, ts=ts, nx=nx, x_trainable=True)], seq_len)
train_set.append(Experiment(u_train_yuqi, y_train_yuqi, ts=ts, nx=nx, x_trainable=True))
test_set = ExperimentsDataset([Experiment(u_test, y_test, ts=ts, nx=nx)], seq_len)
test_set.append(Experiment(u_test_yuqi, y_test_yuqi, ts=ts, nx=nx, x_trainable=False))

TypeError: 'type' object is not subscriptable

**Modèle d'état neuronal Grnssm**
Pas le modèle le plus plus général qui soit puisque pas de terme direct mais on va dire que ça suffit.



$\begin{array}{ccc}
    \dot{x} &=& Ax + Bu + f(x,u) \\
    y &=& Cx + h(x)
    \end{array}$


où $f,h$ sont des MLP.

Si votre système est proche du linéaire : ssest (la fonction matlab) renvoie un fit au-dessus 65% à la louche.

Alors la partie linéaire est utile et vous pouvez l'initialiser avec ce qui sort de Matlab.

Sinon le biais introduit par le linéaire risque de desservir l'apprentissage.

Si votre fit linéaire est grand ~ 80% alors elle peut être gelée (on n'entraîne pas les poids). 

Vous gérez la partie linéaire en la gelant à 0 avec la méthode init_weights()


In [ ]:
import torch
from ctrl_nmod.models.ssmodels.grnssm import Grnssm

nu, ny, nh = 2, 1, 8
# actF = 'relu'
model = Grnssm(nu, ny, nx, nh)
# A0, B0, C0, D0 = findBLA(u_train, y_train, nx, float(1/fs[0]), model_type='continuous')
A0 = -torch.eye(nx)
B0 = torch.Tensor([[0, 1], [1, 0]])
C0 = torch.Tensor([[1, 0]])

model.init_weights_(A0, B0, C0, isLinTrainable=False)

**Neural ODEs**

C'est juste l'ajout d'un intégrateur différentiable sur le modèle d'état.

Pour l'instant 2 solveurs sont implémentés Runge-Kutta 4 et 45 à pas fixe donc c'est ODE45 à pas fixe.

In [2]:
from ctrl_nmod.integrators.integrators import RK4Simulator
sim_model = RK4Simulator(model, ts=torch.Tensor([ts]))

NameError: name 'model' is not defined

**Remarque importante sur l'initalisation:**

Il faut nécessairement que votre $A_0$ soit stable (s'il existe) sinon vous ne pourrez pas entraîner votre modèle.

Il est aussi possible que vous ayez des problèmes de stabilité du schéma d'intégration : 

le modèle continu est stable mais diverge à cause de l'intégration.

Ca arrive ! Une solution est notamment d'utiliser une stratégie d'initialisation des poids
en fonction du schéma.

C'est encore un truc à implémenter dans ma TODO :)

Sinon les autres solutions sont :
* diminuer le pas donc la période d'échantillonage des données
* trouver une initialisation qui donne un truc pas trop mal : l'initialisation par le linéaire en général suffit
* utiliser le paramètre $alpha$ dans le constructeur du Grnssm il impose une borne sur la partie réelle des valeurs propres de A.

$ \Re(\lambda_i(A)) \leq -\alpha$

Mais ça coûte une peu plus en calcul.

**Losses et régularizations**

Plusieurs loss sont implémentées uniquement de la régression.

Chaque loss peut accepter une liste de termes de régularisaition :

$\mathcal{L} = MSE + \sum_{i=1}^{N_{reg}} \lambda_i \mathcal{R}_i$

Les $\lambda_i$ sont les pondérations des régularisations. Ils sont mis à jour si la loss stagne plus de patience_soft **epochs testées**

In [ ]:
import os
from ctrl_nmod.losses.losses import MSELoss, NMSELoss
from ctrl_nmod.regularizations.regularizations import StateRegularization
loss = MSELoss([StateRegularization(model, 0.01, 0.0, updatable=False, verbose=True)])
val_loss = NMSELoss()

**Entraînement**

La classe SSTrainer inclus une méthode fit_ qui fait l'entraînement du modèle.

La seule particularité est qu'une époque est réalisée lorsque l'on a parcouru tout le jeu de données.

Ce qui veut dire tout les points de toutes les expériences.

On évalue aussi la performance sur le jeu de test en simulation sur tout le jeu de données ce qui prend du temps.
Un conseil est donc de mettre test_freq à une valeur plus grande.

In [ ]:
from ctrl_nmod.plot.plots import plot_yTrue_vs_error
from scipy.io import savemat
from ctrl_nmod.train.train import SSTrainer

trainer = SSTrainer(sim_model, loss=loss, val_loss=val_loss)

# Training options

batch_size, lr, keep_best = 128, 1e-3, True
epochs, optimizer = 100, 'adamw'

scheduled = True
step_sched = 0.1
save_path = f'results/try_1/{str(trainer)}'
os.makedirs(save_path, exist_ok=True)

best_model, res = trainer.fit_(train_set=train_set, test_set=test_set,
                               batch_size=batch_size, lr=lr, keep_best=keep_best,
                               save_path=save_path, epochs=epochs, opt=optimizer,
                               scheduled=True)


# Best model simulation on test set
x_sim, y_sim = best_model.simulate(test_set.experiments[0].u, torch.zeros(nx))

os.makedirs(save_path, exist_ok=True)

plot_yTrue_vs_error(test_set.experiments[0].y, y_sim, save_path + 'test_exp_1_sim.png')


savemat(save_path + '/results.mat', res)

torch.save(best_model, save_path + '/model.pkl')
